<a href="https://colab.research.google.com/github/AshwinKotgire/Play_with_LLMs/blob/main/Llamav2_7B_first_look/Llama_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#POC for Llama-v2 For a Basic ChatBbot
##Installing required libraries
1. transformers: Hugging face libraries for loading pretrained transformer checkpoints.
2.accelerate : Manages communications between CPU and GPU more efficiently.
3.datasets : For loading datasets from huggingface for future fine tuning.
4.bitsandbytes : Used for quantization of model to run on limited resources.
5.einops :Einstein-Inspired Notation for operations
6.wandb :weights and biases for visualizations

In [1]:
!pip install -q -U trl transformers accelerate
!pip install -q datasets bitsandbytes einops wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.1/88.1 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.1/519.1 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 42.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 47.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from huggingface_hub import login
login()

##Imports
1. Pipeline : High level pipeline to interact with huggingface models
2. torch : Deeplearning framework used
3. transformers: Huggingface library to interact with transformer models.

In [3]:
# Use a pipeline as a high-level helper
from transformers import pipeline
import torch
import transformers


####BY DIRECTLY USING PIELINE

In [ ]:
model="meta-llama/Llama-2-7b-chat-hf"
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16)

####From transformers import
1. AutoModelForCausalLM : Instantiates one of the model classes of the library (with a causal language modeling head) from a configuration.
2. AutoTokenizer : Tokenizer for a selected model
3. BitsAndBytesConfig : returns object which is used to change datatype / quantise a model


In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_name = "meta-llama/Llama-2-7b-chat-hf"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

#####Testing the chat model
Steps:
1. Prompt : list of prompts to be passed to model
2. inputs : tokenization of prompt

 2.1 adding padding =True if multiple input sequences are of different lenghts
3. generate_ids: gnerated ids of model for a given list of prompts
4. using tokenizer.batch_decode to decode the genrated ids into human readable sentences.

In [11]:
from IPython.display import Markdown

In [18]:
if torch.cuda.is_available():
    print('cuda')
    device = torch.device("cuda")  # If GPU is available, use it.
else:
    device = torch.device("cpu")   # If GPU is not available, use the CPU.


cuda


In [46]:
context="""In 1828, Carl Friedrich Gauss proved his Theorema Egregium (remarkable theorem \
 in Latin), establishing an important property of surfaces. Informally, the theorem says that \
 the curvature of a surface can be determined entirely by measuring distances along paths \
 on the surface. That is, curvature does not depend on how the surface might be embedded in \
 3-dimensional space. See Differential geometry of surfaces. Bernhard Riemann extended Gauss's \
 theory to higher-dimensional spaces called manifolds in a way that also allows distances and angles \
 to be measured and the notion of curvature to be defined, again in a way that is intrinsic to the manifold \
 and not dependent upon its embedding in higher-dimensional spaces. Albert Einstein used the theory of pseudo-Riemannian \
 manifolds (a generalization of Riemannian manifolds) to develop his general theory of relativity. \
 In particular, his equations for gravitation are constraints on the curvature of spacetime."""
instruction='Summarize the context in 10 words.'
prompt=f"""You are given context and you are to forllow the instruction if answer is available in the context, else politely decline. \n
Context: {context} \n
Instruction: {instruction}"""
Markdown(prompt)

You are given context and you are to forllow the instruction if answer is available in the context, else politely decline. 

Context: In 1828, Carl Friedrich Gauss proved his Theorema Egregium (remarkable theorem  in Latin), establishing an important property of surfaces. Informally, the theorem says that  the curvature of a surface can be determined entirely by measuring distances along paths  on the surface. That is, curvature does not depend on how the surface might be embedded in  3-dimensional space. See Differential geometry of surfaces. Bernhard Riemann extended Gauss's  theory to higher-dimensional spaces called manifolds in a way that also allows distances and angles  to be measured and the notion of curvature to be defined, again in a way that is intrinsic to the manifold  and not dependent upon its embedding in higher-dimensional spaces. Albert Einstein used the theory of pseudo-Riemannian  manifolds (a generalization of Riemannian manifolds) to develop his general theory of relativity.  In particular, his equations for gravitation are constraints on the curvature of spacetime. 

Instruction: Summarize the context in 10 words.

In [36]:
prompt="""Use the given question to guide your summary about the context. If you don't know the answer, just say that you don't know, don't try to make up an answer.
Question: Summarize Topic X
Context:
When talking about Topic X, Scenario Y is always referred to. This is due to the relation of

Topic X is a broad topic which covers many aspects of life.

No one knows when Topic X became a thing, its origin is unknown even to this day.

Summary:"""

In [47]:
prompt=f"""<s>[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>

{prompt} [/INST]
"""
print(prompt)

<s>[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>

You are given context and you are to forllow the instruction if answer is available in the context, else politely decline. 

Context: In 1828, Carl Friedrich Gauss proved his Theorema Egregium (remarkable theorem  in Latin), establishing an important property of surfaces. Informally, the theorem says that  the curvature of a surface can be determined entirely by measuring distances along paths  on the surface. That is, curvature does not depend on how the sur

In [48]:
# prompt = 'I liked "Breaking Bad" and "Band of Brothers". Do you have any recommendations of other shows I might like?'
inputs = tokenizer(prompt, return_tensors="pt",padding =True).to(device)
generate_ids = model.generate(inputs.input_ids, max_length=500,top_k=5,top_p=0.5,temperature=0)
Markdown("###"+tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0])
# print(tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0])

###[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>

You are given context and you are to forllow the instruction if answer is available in the context, else politely decline. 

Context: In 1828, Carl Friedrich Gauss proved his Theorema Egregium (remarkable theorem  in Latin), establishing an important property of surfaces. Informally, the theorem says that  the curvature of a surface can be determined entirely by measuring distances along paths  on the surface. That is, curvature does not depend on how the surface might be embedded in  3-dimensional space. See Differential geometry of surfaces. Bernhard Riemann extended Gauss's  theory to higher-dimensional spaces called manifolds in a way that also allows distances and angles  to be measured and the notion of curvature to be defined, again in a way that is intrinsic to the manifold  and not dependent upon its embedding in higher-dimensional spaces. Albert Einstein used the theory of pseudo-Riemannian  manifolds (a generalization of Riemannian manifolds) to develop his general theory of relativity.  In particular, his equations for gravitation are constraints on the curvature of spacetime. 

Instruction: Summarize the context in 10 words. [/INST]
Gauss proved Theorema Egregium, Riemann extended it to manifolds, Einstein used it for general relativity.

In [12]:
Markdown('Hello \n\n')

Hello 



In [ ]:
tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[1]

'How to quickly breathe?\n\nBreathing is a vital function that helps to bring oxygen into the body and remove carbon dioxide. Here are some tips on how to quickly breathe:\n\n1. Take Deep Breaths: When you\'re feeling stressed or anxious, your breathing can become shallow and rapid. To calm down, try taking deep breaths through your nose and exhaling through your mouth. Inhale for a count of four, hold your breath for a count of four, and exhale for a count of four.\n\n2. Practice Diaphragmatic Breathing: The diaphragm is a muscle that helps you breathe more efficiently. To practice diaphragmatic breathing, place one hand on your stomach and the other on your chest. Inhale through your nose, allowing your stomach to rise as your diaphragm descends. Exhale through your mouth, allowing your stomach to fall as your diaphragm rises.\n\n3. Use Your Vocal Cords: When you\'re feeling anxious or stressed, try using your vocal cords to help you breathe more deeply. Take a deep breath in through

How to quickly breathe?

Breathing is a vital function that helps to bring oxygen into the body and remove carbon dioxide. Here are some tips on how to quickly breathe:

1. Take Deep Breaths: When you're feeling stressed or anxious, your breathing can become shallow and rapid. To calm down, try taking deep breaths through your nose and exhaling through your mouth. Inhale for a count of four, hold your breath for a count of four, and exhale for a count of four.

2. Practice Diaphragmatic Breathing: The diaphragm is a muscle that helps you breathe more efficiently. To practice diaphragmatic breathing, place one hand on your stomach and the other on your chest. Inhale through your nose, allowing your stomach to rise as your diaphragm descends. Exhale through your mouth, allowing your stomach to fall as your diaphragm rises.

3. Use Your Vocal Cords: When you're feeling anxious or stressed, try using your vocal cords to help you breathe more deeply. Take a deep breath in through your nose, and then exhale slowly through your mouth, making a "hmm" sound. This can help you relax and focus on your breathing.

4. Try Box Breathing: Box breathing is a simple technique that can help you breathe more deeply and evenly. To practice box breathing, inhale for a count of four, hold your breath for a count of four, exhale for a count of four, and then hold your breath again for a count of four. Repeat this pattern several times.

5. Practice Progressive Muscle Relaxation: Progressive muscle relaxation is a technique that can help you relax and focus on your breathing. To practice progressive muscle relaxation, start by tensing and relaxing different muscle groups in your body, such as your toes, calves, and fingers. As you tense each muscle group, hold your breath for a count of four, and then release and exhale slowly.

6. Use Visualization Techniques: Visualization can help you